# Logical S Gate

This notebook demonstrates the logical S gate with inplace Y basis measurement and gate teleportation [<cite data-footcite-t="Gidney_inplace_access_2024"></cite>].

## Construction

S gate can be implemented by gate teleportation:

- $M_{ZZ}$ measurement between the target qubit and an ancilla qubit initialized in the $|+\rangle$ state.
- Y basis measurement of the ancilla qubit.

`tqec` provides builtin functions `tqec.gallery.s_gate_teleportation` to construct it.

In [ ]:
from tqec import PauliBasis
from tqec.gallery import s_gate_teleportation

graph = s_gate_teleportation(PauliBasis.Y)
graph.view_as_html()

S gate realizes $Y \rightarrow X$ and $Z \rightarrow Z$ stabilizer flows, ignoring the signs. The following correlation surface shows that the computation maps the $Y$ operator at the bottom input to the $X$ operator at the top output.

In [ ]:
correlation_surfaces = graph.find_correlation_surfaces()
graph.view_as_html(
    pop_faces_at_directions=("-Y",),
    show_correlation_surface=correlation_surfaces[0],
)

## Circuit

You can download the circuit for a $d=3$ S gate circuit from [here](../media/gallery/memory/circuit.stim), or generate it with the code below. You can also open the circuit in [Crumble]().

In [ ]:
from tqec import NoiseModel, compile_block_graph

compiled_graph = compile_block_graph(graph)
circuit = compiled_graph.generate_stim_circuit(
    k=1, noise_model=NoiseModel.uniform_depolarizing(p=0.001)
)

## Simulation

Here we show the simulation results of $Y \rightarrow X$ and $Z \rightarrow Z$ stabilizer flows under **uniform depolarizing** noise mode.

<details><summary><strong>Click to show the full code used for simulation</strong></summary>

```py
from multiprocessing import cpu_count
from pathlib import Path

import matplotlib.pyplot as plt
import numpy
import sinter

from tqec import NoiseModel
from tqec.gallery.s_gate_teleportation import s_gate_teleportation
from tqec.simulation.plotting.inset import plot_observable_as_inset
from tqec.simulation.simulation import start_simulation_using_sinter
from tqec.utils.enums import PauliBasis

SAVE_DIR = Path("results")


def generate_graphs(in_observable_basis: PauliBasis) -> None:
    block_graph = s_gate_teleportation(in_observable_basis)
    zx_graph = block_graph.to_zx_graph()

    correlation_surfaces = block_graph.find_correlation_surfaces()

    stats = start_simulation_using_sinter(
        block_graph,
        range(1, 4),
        list(numpy.logspace(-4, -1, 10)),
        NoiseModel.uniform_depolarizing,
        manhattan_radius=2,
        observables=correlation_surfaces,
        num_workers=cpu_count(),
        max_shots=1_000_000,
        max_errors=5_000,
        decoders=["pymatching"],
        # note that save_resume_filepath and database_path can help reduce the time taken
        # by the simulation after the database and result statistics have been saved to
        # the chosen path
        save_resume_filepath=Path(
            f"../_examples_database/s_gate_stats_{in_observable_basis.value}.csv"
        ),
        database_path=Path("../_examples_database/database.pkl"),
    )

    for i, stat in enumerate(stats):
        fig, ax = plt.subplots()
        sinter.plot_error_rate(
            ax=ax,
            stats=stat,
            x_func=lambda stat: stat.json_metadata["p"],
            failure_units_per_shot_func=lambda stat: stat.json_metadata["d"],
            group_func=lambda stat: stat.json_metadata["d"],
        )
        plot_observable_as_inset(ax, zx_graph, correlation_surfaces[i])
        ax.grid(axis="both")
        ax.legend()
        ax.loglog()
        ax.set_title("Logical S Gate Error Rate")
        ax.set_xlabel("Physical Error Rate")
        ax.set_ylabel("Logical Error Rate(per round)")
        fig.savefig(
            SAVE_DIR
            / f"logical_s_gate_result_input_{in_observable_basis}_operator.png"
        )


def main():
    SAVE_DIR.mkdir(exist_ok=True)
    generate_graphs(PauliBasis.Z)
    generate_graphs(PauliBasis.X)


if __name__ == "__main__":
    main()

```

</details>

In [ ]:
from multiprocessing import cpu_count
from pathlib import Path

import matplotlib.pyplot as plt
import numpy
import sinter

from tqec import NoiseModel
from tqec.gallery.s_gate_teleportation import s_gate_teleportation
from tqec.simulation.plotting.inset import plot_observable_as_inset
from tqec.simulation.simulation import start_simulation_using_sinter
from tqec.utils.enums import PauliBasis


def generate_graphs(in_observable_basis: PauliBasis) -> None:
    """Generate the logical error-rate graphs corresponding to the provided basis."""
    block_graph = s_gate_teleportation(in_observable_basis)
    zx_graph = block_graph.to_zx_graph()

    correlation_surfaces = block_graph.find_correlation_surfaces()

    stats = start_simulation_using_sinter(
        block_graph,
        range(1, 4),
        list(numpy.logspace(-4, -1, 10)),
        NoiseModel.uniform_depolarizing,
        manhattan_radius=2,
        observables=correlation_surfaces,
        num_workers=cpu_count(),
        max_shots=1_000_000,
        max_errors=5_000,
        decoders=["pymatching"],
        # note that save_resume_filepath and database_path can help reduce the time taken
        # by the simulation after the database and result statistics have been saved to
        # the chosen path
        save_resume_filepath=Path(
            f"../_examples_database/s_gate_stats_{in_observable_basis.value}.csv"
        ),
        database_path=Path("../_examples_database/database.pkl"),
    )

    for i, stat in enumerate(stats):
        fig, ax = plt.subplots()
        sinter.plot_error_rate(
            ax=ax,
            stats=stat,
            x_func=lambda stat: stat.json_metadata["p"],
            failure_units_per_shot_func=lambda stat: stat.json_metadata["d"],
            group_func=lambda stat: stat.json_metadata["d"],
        )
        plot_observable_as_inset(ax, zx_graph, correlation_surfaces[i])
        ax.grid(axis="both")
        ax.legend()
        ax.loglog()
        ax.set_title("Logical S Gate Error Rate")
        ax.set_xlabel("Physical Error Rate")
        ax.set_ylabel("Logical Error Rate(per round)")

### $Y \rightarrow X$

In [ ]:
generate_graphs(PauliBasis.Y)

### $Z \rightarrow Z$

In [ ]:
generate_graphs(PauliBasis.Z)

## References

.. footbibliography::